In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import JSONLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings

In [4]:
from IPython.display import display, Markdown
import json
from pathlib import Path
from pprint import pprint

In [30]:
file = '/home/boringtao/Projects/Math_Agent/notebooks/solutions.csv'
loader = CSVLoader(file_path=file)

In [31]:
data = loader.load()

In [32]:
print(data)

[Document(page_content='Problem: Classes A and B start from the school at the same time to go to a park that is 51 km away. Class A walks at a speed of 4 km/h, and Class B walks at a speed of 5 km/h. The school has a car that can travel at 60 km/h and can carry one class of students at a time. To ensure that both classes arrive at the park in the shortest time possible, how many kilometres do Classes A and B need to walk?\nAnswer: Let Class A walk X km. Then the time Class A takes to walk is X/4 hours. In this time, the car travels 15X km. The car takes Class B to the point where they should start walking and then returns to meet Class A. The distance the car travels to take Class B is (15X+X)=8X. Therefore, Class B walks 51−8X km, and the time Class B takes to walk is 51−8X hours. In this time, the car travels (51−8X)×12 km. At this point, the car needs to pick up Class A and return to the park.\nSolution: By solving the equation (51−8X)×12+(51−8X) +X=51, we get X=5.5 and 51−8X=7. The

In [34]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [55]:
query ="Please list all the problems involving reasoning."

In [56]:
response = index.query(query)

In [57]:
display(Markdown(response))

 The problems involving reasoning are:
1. A company organises a social event. 50% of the sales department participates, and 60% of the IT department participates. The number of people from the IT department who participate is 6 more than the number from the sales department. Later, due to urgent work, 60% of the remaining people in the sales department and 50% of the remaining people in the IT department are selected to form a 30-person work team. How many employees does the company have?
2. Classes A and B start from the school at the same time to go to a park that is 51 km away. Class A walks at a speed of 4 km/h, and Class B walks at a speed of 5 km/h. The school has a car that can travel at 60 km/h and can carry one class of students at a time. To ensure that both classes arrive at the park in the shortest time possible, how many kilometres do Classes A and B need to walk?

In [59]:
embeddings = OpenAIEmbeddings()

In [61]:
db = DocArrayInMemorySearch.from_documents(
    data, 
    embeddings
)

In [62]:
docs = db.similarity_search(query)

In [63]:
len(docs)

2

In [64]:
retriever = db.as_retriever()

In [65]:
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [66]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [67]:
response = llm.call_as_llm(f"{qdocs} Question: Please convert the problems into equations.") 

In [68]:
display(Markdown(response))

Problem 1:
Let X be the distance Class A walks.
The time Class A takes to walk is X/4 hours.
In this time, the car travels 15X km.
The car takes Class B to the point where they should start walking and then returns to meet Class A.
The distance the car travels to take Class B is (15X+X)=8X.
Therefore, Class B walks 51−8X km, and the time Class B takes to walk is 51−8X hours.
In this time, the car travels (51−8X)×12 km.
At this point, the car needs to pick up Class A and return to the park.

Equation: (51−8X)×12 + (51−8X) + X = 51

Problem 2:
Let X be the number of people in the sales department and Y be the number of people in the IT department.
The number of people participating from the sales department is 0.5X.
The number of people participating from the IT department is 0.6Y.
The IT department has 6 more people participating than the sales department, so 0.6Y−0.5X=6.
The remaining number of people from the sales department is 0.5X.
60% of the remaining people are selected, which is 0.3X.
The remaining number of people from the IT department is 0.4Y.
50% of the remaining people are selected, which is 0.2Y.
Therefore, 0.3X+0.2Y=30.

Equations: 
0.6Y−0.5X=6
0.3X+0.2Y=30